## Maxwell A. Fine 2024-07-22

### This notebook is for:
* Learning how to record baseband data with `vrt_to_sigmf`
* Slicing to only keep baseband data for 1 second by a "good" `.h5` candidate
* Slicing to only keep baseband data for 1 second in the other band for DM dependent arrival time
* starting to integreate into the pipeline


### Notes:
* 1 second might end up being too much data to keep
* Maybe calculate width of arrivial times for the band based on DM and keep 2-3x that amount
* We will write the baseband data to a 1 tb ram disk `/data_tmp`
* No easy way to record at **EXACTLY** the same time as the `.fil` data  


### '--help' option
```
(venv) frb@uranus:/home_local/frb/git/vrt-iq-tools$ vrt_to_sigmf --help
VRT samples to file. Allowed options:
  --help                     help message
  --file arg (=vrt_samples)  name of the file to write binary samples to
  --auto-file arg            prefix of the auto generated filename to write 
                             binary samples to
  --nsamps arg (=0)          total number of samples to receive
  --duration arg (=0)        total number of seconds to receive
  --progress                 periodically display short-term bandwidth
  --channel arg (=0)         which channel(s) to use (specify "0", "1", "0,1", 
                             etc)
  --author arg               core:author in sigmf-meta
  --description arg          core:description in sigmf-meta
  --int-second               align start of reception to integer second
  --null                     run without writing to file
  --continue                 don't abort on a bad packet
  --meta-only                only create sigmf-meta file
  --dt-trace                 add DT trace data
  --vrt                      write VRT stream to file
  --address arg (=localhost) VRT ZMQ address
  --zmq-split                create a ZeroMQ stream per VRT channel, increasing
                             port number for additional streams
  --instance arg (=0)        VRT ZMQ instance
  --port arg                 VRT ZMQ port
  --hwm arg (=10000)         VRT ZMQ HWM


This application streams data from a VRT stream to a file.
```


### Trying this out in the terminal `vrt_to_sigmf --file max_test_file --dt-trace --instance 2 `:
* `--file` controls data file & json file name
* not sure what the `dt-trace` is or does, but we need it
* `--instance` controls what band we are looking at, L1 and L2 use different ones but PH, and PV use the same? - look into


### Open Questions:
* Is Time of the candiate the non-DM corrected time at the highest freq in the band?  


### Steps:
* [x] try saving baseband file via CLI
* [x] subprocess call to save baseband
* [x] slice baseband segment and update meta data, write to new file
* [ ] Have function to to adjust time of arrival for DM
* [ ] look into what time is listed for a time candidate: top, bottom, center of band? corrected for DM?
* [ ] write something to slice the corresponding region of L or P bands for a candidate
* [ ] delete original baseband data after a loop


### Lets make a subprocess function

In [4]:
import subprocess
import os

def run_vrt_to_sigmf(file_name, instance, duration, ram_data_dir = '/data_tmp'):
    # Define the command and arguments

    # Define the output file path
    output_file_path = os.path.join(ram_data_dir, file_name)
    print(output_file_path)
                     
    command = [
        "vrt_to_sigmf",
        "--file", output_file_path,
        "--dt-trace", # removed for testing, but needs to be uncommented when recording with the telescope
        "--instance", str(instance),
        "--duration", str(duration),
        "--progress",
    ]
    
    try:
        # Start the process
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        # Print stdout and stderr as the process runs
        for stdout_line in iter(process.stdout.readline, ''):
            if stdout_line:
                print(f"Output: {stdout_line.strip()}")
        for stderr_line in iter(process.stderr.readline, ''):
            if stderr_line:
                print(f"Error: {stderr_line.strip()}")
        
        # Ensure all output has been processed
        process.stdout.close()
        process.stderr.close()
        
        # Wait for the process to finish and get the return code
        return_code = process.wait()
        
        print(f"Return Code: {return_code}")
        
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return e.returncode

### Example use

In [5]:
# example use
file_name = "max_test_filev3"
ram_data_dir = '/data_tmp'
instance = 2
duration = 8
run_vrt_to_sigmf(file_name, instance, duration)

/data_tmp/max_test_filev3
Output: Press Ctrl + C to stop receiving...
Output: # First frame: 10000 samples, 1721944731 full secs, 0.803711040 frac secs (counter 10)
Output: # VRT Context:
Output: #    Stream ID (channel): 1 (0)
Output: #    Sample Rate [samples per second]: 100000000
Output: #    RF Freq [Hz]: 1250000000
Output: #    Bandwidth [Hz]: 80000000
Output: #    Gain [dB]: 70
Output: #    Ref lock: external
Output: #    Time cal: pps
Output: #    Cal time: 1721937166
Output: 100.024 Msps, 47% I (8 of 16 bits), 0% I clip,
Output: 99.9997 Msps, 46% I (8 of 16 bits), 0% I clip,
Output: 99.9991 Msps, 47% I (9 of 16 bits), 0% I clip,
Output: 100 Msps, 47% I (9 of 16 bits), 0% I clip,
Output: 100.004 Msps, 47% I (8 of 16 bits), 0% I clip,
Output: 99.9996 Msps, 47% I (8 of 16 bits), 0% I clip,
Output: 99.999 Msps, 47% I (9 of 16 bits), 0% I clip,
Return Code: 0


### lets try reading the data, and json meta file in 

In [6]:
import sigmf
import numpy as np
file_path = os.path.join(ram_data_dir, file_name)

handle = sigmf.sigmffile.fromfile(file_path) # this is the json stuff
iq = handle.read_samples(); # this is the data, numpy array, complex 64 float shape (n samples,)
sample_rate = int(handle.get_global_info()[handle.SAMPLE_RATE_KEY])
center_freq = handle.get_capture_info(0)["core:frequency"]
start_time = handle.get_capture_info(0)["core:datetime"]
#version = handle.get"core:version": "0.0.1"  # Add the core:version property
version = handle.get_global_info()[handle.VERSION_KEY] #"core:version": "1.0.0",

In [20]:
iq.dtype

dtype('complex64')

In [7]:
version

'1.0.0'

In [8]:
print(start_time, '\n', handle)

2024-07-25T21:58:51.803711 
 {
    "global": {
        "core:datatype": "ci16_le",
        "core:recorder": "vrt_to_sigmf",
        "core:sample_rate": 100000000,
        "core:sha512": "532d88a4b24518f4e9aea3396578011861314cca0dff29496613024b62dca07fc7b78a93d9720d3185b2750084aa3e8644975045d4f2e459d75b551c47c47b3c",
        "core:version": "1.0.0",
        "dt:datetime": "2024-07-25T21:58:51.900000",
        "dt:focusbox_position_mm": 451,
        "dt:pointing:active_tracker": "j2000tracker",
        "dt:pointing:az_speed_deg_s": -0.002912,
        "dt:pointing:current:az_deg": -100.178,
        "dt:pointing:current:dec_deg": 32.266,
        "dt:pointing:current:el_deg": 34.017,
        "dt:pointing:current:ra_h": 23.639,
        "dt:pointing:dt_model": "true",
        "dt:pointing:dt_model_j2000": "true",
        "dt:pointing:el_speed_deg_s": 0.001443,
        "dt:pointing:error:az_deg": 0.0,
        "dt:pointing:error:el_deg": -0.0,
        "dt:pointing:model:a0": 0.0,
        "dt:po

### Lets try cutting out and saving a slice

In [9]:
from datetime import datetime, timedelta
import sigmf
import numpy as np
import os

import os
import sigmf

def read_baseband_data(ram_data_dir, orig_file_name, return_data=False):
    """
    Read the metadata from a baseband file and optionally return the data.

    Parameters:
    ram_data_dir (str): Directory containing the baseband file.
    orig_file_name (str): Name of the baseband file.
    return_data (bool): Whether to return the IQ data as well. Defaults to False.

    Returns:
    tuple: A tuple containing:
        - sample_rate (int): Sample rate in Hz.
        - center_freq (float): Center frequency in Hz.
        - start_time (str): Start time in ISO format.
        - version (str): Version of the SigMF file.
        - iq (np.ndarray): IQ data as a numpy array, complex 64 float shape (n samples,) if return_data is True.
    """
    file_path = os.path.join(ram_data_dir, orig_file_name)
    
    # Read the SigMF file
    handle = sigmf.sigmffile.fromfile(file_path)
    sample_rate = int(handle.get_global_info()[handle.SAMPLE_RATE_KEY])
    center_freq = handle.get_capture_info(0)["core:frequency"]
    start_time = handle.get_capture_info(0)["core:datetime"]
    version = handle.get_global_info()[handle.VERSION_KEY]  # "core:version": "1.0.0"

    if return_data:
        iq = handle.read_samples()  # Read the data, numpy array, complex 64 float shape (n samples,)
        return sample_rate, center_freq, start_time, version, iq

    return sample_rate, center_freq, start_time, version



def slice_and_save_baseband_data(ram_data_dir, orig_file_name, 
                              out_data_dir, new_file_name, start_s, delta_t=1):
    """
    Slices a portion of IQ data from a SigMF file and saves the new sliced data along with updated metadata to new files.

    Parameters:
    ram_data_dir (str): Directory where the original SigMF file is stored.
    orig_file_name (str): Name of the original SigMF file to process.
    out_data_dir (str): Directory where the new sliced SigMF files will be saved.
    new_file_name (str): Base name for the new SigMF files to be saved.
    start_s (float): Start time in seconds from the beginning of the observation.
    delta_t (float): Duration of the slice in seconds. Default is 1.

    Returns:
    None

    The function reads the SigMF file, slices the IQ data between start_s and start_s + delta_t seconds, updates the start time,
    and saves the new sliced data and metadata to new files in the specified output directory.

    Example:
    slice_and_save_sigmf_data('/path/to/orig_dir', 'example.sigmf-meta', '/path/to/out_dir', 'new_example', start_s=2, delta_t=1)
    """

    # Call the read_baseband_data function
    sample_rate, center_freq, start_time, version, iq = read_baseband_data(ram_data_dir, 
                                                                           orig_file_name, return_data=True)

    # indexs in data
    start = int(start_s * sample_rate)
    stop = int((start_s + delta_t) * sample_rate)

    # slice data
    data_slice = iq[start:stop]
    # update start time for the slice
    new_start_time = datetime.fromisoformat(start_time) + timedelta(seconds=start_s)
    new_start_time_str = new_start_time.isoformat()

    # Create a new SigMF file
    new_handle = sigmf.sigmffile.SigMFFile()

    # Update the metadata
    new_handle.set_global_info({
        "core:sample_rate": sample_rate,
        "core:datatype": handle.get_global_info()["core:datatype"],
        "core:version": version  # Add the core:version property
    })

    # Set the capture information directly
    new_handle.captures = [{
        "core:frequency": center_freq,
        "core:datetime": new_start_time_str
    }]

    # Set the annotation information
    new_handle.annotations = [{
        "core:sample_count": len(data_slice)
    }]

    # Define the new file paths
    new_meta_file_path = os.path.join(out_data_dir, f"{new_file_name}.sigmf-meta")
    new_data_file_path = os.path.join(out_data_dir, f"{new_file_name}.sigmf-data")

    # Save the new SigMF metadata file
    new_handle.tofile(new_meta_file_path)

    # Save the new IQ data to a binary file
    data_slice.tofile(new_data_file_path)

    print(f"New files saved to: {new_meta_file_path} and {new_data_file_path}")




### Now lets write some functions to account for DM arrivial time delay

This is based on `candidate_maker.py` (Tammo Jan Dijkema) which is itself based on `Candidate.ipynb` from `your` documentation

In [11]:
### function to determine file size (aka duration) Fast

import os
import numpy as np

def get_number_of_samples_from_sigmf(filepath, sample_dtype=np.complex64, file_extension='.sigmf-data'):
    """
    Calculate the number of samples in a SigMF binary data file.

    This function reads the size of the binary data file associated with the given
    SigMF filepath and calculates the number of samples based on the specified
    sample data type.

    Parameters:
    ----------
    filepath : str
        The base filepath (without the extension) of the SigMF file. The function
        expects the binary data file to have a '.sigmf-data' extension.
    sample_dtype : numpy.dtype, optional
        The data type of the samples in the binary file. Default is np.complex64.
    file_extension : str, optional 
        File extension to add, default is '.sigmf-data'

    Returns:
    -------
    int
        The number of samples in the binary data file.
    """
    base, ext = os.path.splitext(filepath)
    data_file = base + file_extension
    file_size = os.path.getsize(data_file)
    sample_size = np.dtype(sample_dtype).itemsize
    num_samples = file_size // sample_size
    return num_samples

data_file = file_path 
num_samples = get_number_of_samples_from_binary(data_file,)
print(f"Number of samples: {num_samples}")

Number of samples: 400000000


In [1]:
# Inital functions copied from `candidate_maker.py`

def time_from_dm(f_mhz, dm):
    """
    Calculate the dispersion (DM) delay time in seconds for a given frequency and dispersion measure (DM).

    DM constant is 4149 for freq units in mhz , and dm in pc/cm^3

    Parameters:
    f_mhz (float): Frequency in megahertz (MHz).
    dm (float): Dispersion measure in pc cm^-3.

    Returns:
    float: Time delay in seconds for a given freq and DM.
    """
    return 4149 * dm / f_mhz**2


def get_mjd(mjd1, freq1_mhz, freq2_mhz, dm):
    """
    Calculate the Modified Julian Date (MJD) at a second frequency, given a detection at a first frequency.

    Parameters:
    mjd1 (float): MJD of the detection at the first frequency.
    freq1_mhz (float): First frequency in megahertz (MHz) at which the detection was made.
    freq2_mhz (float): Second frequency in megahertz (MHz) at which to calculate the MJD.
    dm (float): Dispersion measure in pc cm^-3.

    Returns:
    float: MJD at the second frequency.
    """
    offset1_s = time_from_dm(freq1_mhz, dm)
    offset2_s = time_from_dm(freq2_mhz, dm)
    print("Correction in seconds:", (offset2_s - offset1_s))
    return mjd1 + (offset2_s - offset1_s) / (24 * 3600)


def compute_time(sigmf_file1, sigmf_file2, dm, time_candidate):
    """
    Compute the time offset in seconds from the start of the baseband data to a given MJD and frequency.

    Parameters:
    sigmf_file1 (str): Path to the first SigMF file.
    sigmf_file2 (str): Path to the second SigMF file.
    dm (float): Expected Dispersion measure in pc cm^-3.
    time_candidate (float): Candidate detection time in seconds for file1.

    Returns:
    float: Time offset in seconds from the start of the baseband data for file2, this is time_candidate for file2

    Raises:
    RuntimeError: If the computed time offset is before the start or after the end of the baseband data.
    """
    # Extract directory and file name from sigmf_file1
    baseband_dir1, baseband_file_name1 = os.path.split(sigmf_file1)
    
    # Read metadata from the first SigMF file
    sample_rate1, center_freq1, start_time1, version1 = read_baseband_data(baseband_dir1, baseband_file_name1)

    # Convert start time to MJD
    start_time_dt1 = datetime.fromisoformat(start_time1)
    mjd_start1 = start_time_dt1.toordinal() + 1721424.5

    # Calculate MJD of the candidate event
    mjd_candidate = mjd_start1 + time_candidate / (24 * 3600)

    # Extract directory and file name from sigmf_file2
    baseband_dir2, baseband_file_name2 = os.path.split(sigmf_file2)

    # Read metadata from the second SigMF file
    sample_rate2, center_freq2, start_time2, version2 = read_baseband_data(baseband_dir2, baseband_file_name2)

    # Compute the corrected MJD
    # this is arrival time in the new frequency band
    mjd_corrected = get_mjd(mjd_candidate, center_freq1 / 1e6, center_freq2 / 1e6, dm)

    # Read in start time for file2
    start_time_dt2 = datetime.fromisoformat(start_time2)
    mjd_start2 = start_time_dt2.toordinal() + 1721424.5
    
    # Compute the time offset in seconds from the start of the second SigMF file
    time_offset = (mjd_corrected - mjd_start2) * 24 * 3600

    print(f"Computed time = {time_offset} seconds since start")

    # Calculate the number of samples in the second SigMF file
    n_samples2 = get_number_of_samples_from_sigmf(sigmf_file2, sample_dtype=np.complex64, file_extension='.sigmf-data')

    # Calculate the duration of the data in the second SigMF file
    duration = n_samples2 / sample_rate2

    if time_offset < 0:
        raise RuntimeError("Computed time is before the start of the baseband data.")
    elif time_offset > duration:
        raise RuntimeError("Computed time is after the end of the baseband data.")
    
    return time_offset



### Alright, now we write a few wrappers.

* write a function to get the time_candidate in mjd, using the `.fil` header corresponding to a good candidate to get the start time of the `.fil` file for a specific band


* We want a loop function that parses file names?:
    - loops over and cuts out slices of data, writes to new files from its band, and the other bands using the `time_offset` calculated
    - fail gracefully if the file does not exist, or time_offset is <0, or greater then the duration 
<br><br>


* We want a function to constantly (over)write the baseband data
    - can we read in a partially written file?
    - 10 mins for each band, not at the same exact time as the `.fil` files
    - delete / overwrite after 10 min, can we make this rolling? want to always have 10 min, not 0-10 min cylces



In [58]:
import os
import re

def read_h5_metadata(file_names):
    """
    Extracts metadata from a list of HDF5 file names.

    This function processes each file name in the input list to extract and return the directory, 
    basename, tcand, dm components, band, and fil_file path.
    
    Parameters:
    file_names (list of str): List of HDF5 file names. Each file name is expected to follow the pattern
                              'basename_tcand_<tcand>_dm_<dm>_snr_<snr>.h5'.
    
    Returns:
    tuple: A tuple containing: file_names, (dir_name, basename, tcand, dm, band, fil_file)
           - file_names (list of str): The original list of file names.
           - results (list of tuples): A list of tuples where each tuple contains:
             - dir_name (str): The directory of the file (empty string if no directory is specified).
             - basename (str): The base name of the file up to `_tcand`.
             - tcand (str): The tcand value extracted from the file name.
             - dm (str): The dm value extracted from the file name.
             - band (str): The band value extracted from the file name.
             - fil_file (str): The corresponding fil_file path based on the extracted band.
    """
    results = []
    pattern = re.compile(r'(?P<basename>.+?)_(?P<band>L[0-9]_Band)_tcand_(?P<tcand>[\d.]+)_dm_(?P<dm>[\d.]+)')

    for file_name in file_names:
        # Extract directory and base name
        dir_name = os.path.dirname(file_name)
        match = pattern.search(os.path.basename(file_name))
        if match:
            basename = match.group('basename')
            band = match.group('band')
            tcand = match.group('tcand')
            dm = match.group('dm')
            
            # Construct fil_file path
            fil_file = os.path.join("/data/frb", dir_name.split('/')[2], band, f"{basename}_L1_Band_{dir_name.split('/')[2]}_{tcand.replace('.', '_')}.fil")
            
            results.append((dir_name, basename, tcand, dm, band, fil_file))
    
    return file_names, results

# Example usage
file_names = [
    "/data/frb/2024-07-25/good/FRB20240619D_L2_Band_2024_07_25_03_27_44_tcand_25.1402560_dm_450.5_snr_6.3.h5"
]

print(read_h5_metadata(file_names))


(['/data/frb/2024-07-25/good/FRB20240619D_L2_Band_2024_07_25_03_27_44_tcand_25.1402560_dm_450.5_snr_6.3.h5'], [])


In [51]:
import os
from your.formats.pysigproc import SigprocFile
import re

import os
import re

def read_h5_metadata(file_names):
    """
    Extracts metadata from a list of HDF5 file names.

    This function processes each file name in the input list to extract and return the directory, 
    basename, tcand, and dm components. The basename is the part of the filename up until `_tcand`.
    
    Parameters:
    file_names (list of str): List of HDF5 file names. Each file name is expected to follow the pattern
                              'basename_tcand_<tcand>_dm_<dm>_snr_<snr>.h5'.
    
    Returns:
    tuple: A tuple containing: file_names, (dir_name, basename, tcand, dm)
           - file_names (list of str): The original list of file names.
           - results (list of tuples): A list of tuples where each tuple contains:
             - dir_name (str): The directory of the file (empty string if no directory is specified).
             - basename (str): The base name of the file up to `_tcand`.
             - tcand (str): The tcand value extracted from the file name.
             - dm (str): The dm value extracted from the file name.
    """
    results = []
    pattern = re.compile(r'(?P<basename>.+?)_tcand_(?P<tcand>[\d.]+)_dm_(?P<dm>[\d.]+)')

    for file_name in file_names:
        # Extract directory and base name
        dir_name = os.path.dirname(file_name)
        match = pattern.search(os.path.basename(file_name))
        if match:
            basename = match.group('basename')
            tcand = match.group('tcand')
            dm = match.group('dm')
            results.append((dir_name, basename, tcand, dm))
    
    return file_names, results

def read_fil_metadata(file_path):
    """
    Reads metadata from a .fil file and returns it as a tuple.

    # TODO Max Fine, I want to calculate the duration of the .fil file and return it 
    
    Args:
    file_path (str): The path to the .fil file.
    
    Returns:
    tuple: A tuple containing (tstart_mjd, basename, fch1, foff, nchans, tsamp,)

    fch1 = fil.fch1  # Frequency of the first channel
    foff = fil.foff  # Frequency offset between channels
    nchans = fil.nchans  # Number of channels
    tsamp = fil.tsamp  # sampling interval (seconds)
    nbits = fil.nbits # Number of bits the data are recorded in.
    tstart_mjd = fil.tstart  # Start time in MJD
    """
    fil = SigprocFile(file_path)
    
    basename = os.path.basename(file_path)  # Extracting the basename of the file
    fch1 = fil.fch1  # Frequency of the first channel
    foff = fil.foff  # Frequency offset between channels
    nchans = fil.nchans  # Number of channels
    tsamp = fil.tsamp  # sampling interval (seconds)
    nbits = fil.nbits # Number of bits the data are recorded in.
    tstart_mjd = fil.tstart  # Start time in MJD

    # Assuming fil.data is a numpy array containing the data, its size divided by nchans gives the number of samples
    nsamples = np.shape(fil.get_data) # // nchans
    
    return (tstart_mjd, basename, fch1, foff, nchans, tsamp, nbits, nsamples)

In [55]:
# Example usage
file_names = [
    '/data/frb/2024-07-25/good/' +'FRB20240619D_L2_Band_2024_07_25_03_27_44_tcand_25.1402560_dm_450.5_snr_6.3.h5'
]

print(read_h5_metadata(file_names))

# Example usage
file_path = '/data/frb/2024-07-25/L1_Band/FRB20240722A_L1_Band_2024_07_26_02_03_12.fil'
metadata = read_fil_metadata(file_path)
print(metadata)

(['/data/frb/2024-07-25/good/FRB20240619D_L2_Band_2024_07_25_03_27_44_tcand_25.1402560_dm_450.5_snr_6.3.h5'], [('/data/frb/2024-07-25/good', 'FRB20240619D_L2_Band_2024_07_25_03_27_44', '25.1402560', '450.5')])
(60517.00223379642, 'FRB20240722A_L1_Band_2024_07_26_02_03_12.fil', 1300.0, -0.3125, 320, 0.0001984, 32, ())


In [33]:
help(SigprocFile)

Help on class SigprocFile in module your.formats.pysigproc:

class SigprocFile(builtins.object)
 |  SigprocFile(fp=None, copy_hdr=None)
 |  
 |  Simple functions for reading sigproc filterbank files from python. Not all possible features are implemented.
 |  
 |  Original Source from Paul Demorest's [pysigproc.py](https://github.com/demorest/pysigproc/blob/master/pysigproc.py).
 |  
 |  Args:
 |  
 |      fp (str): file name
 |      copy_hdr (bool): copy header from another SigprocFile class object
 |  
 |  Attributes:
 |  
 |      rawdatafile (str): Raw data file
 |      source_name (str): Source Name
 |      machine_id (int): Machine ID
 |      barycentric (int): If 1 the data is barycentered
 |      pulsarcentric (int): Is the data in pulsar's frame of reference?
 |      src_raj (float): RA of the source (HHMMSS.SS)
 |      src_deg (float): Dec of the source (DDMMSS.SS)
 |      az_start (float): Telescope Azimuth (degrees)
 |      za_start (float): Telescope Zenith Angle (degrees)
 

In [30]:
tstart_mjd

60517.00223379642